<a href="https://colab.research.google.com/github/alwaysneedhelp/AI-Challenge/blob/main/advertisement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Importing Necessary Libraries***

In [127]:
import matplotlib as plt
import seaborn as sns
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.functional as F
from google.colab import drive
from torch.utils.data import DataLoader, Dataset
import torch
import random
import zipfile
import os
from sklearn.model_selection import train_test_split
from PIL import Image
import torchvision.models as models
import torchvision.transforms as T
import torch.optim as optim
from sklearn.metrics import f1_score
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# ***Loading Training Data***

In [115]:
# Fetch the train.csv from mydrive to local colab, so it is easier to work with it

if not (os.path.isfile('train.csv')):
  !cp '/content/drive/MyDrive/AI_Challenge_advertisement_train.csv' train.csv

df = pd.read_csv('train.csv')

In [116]:
# Make the results reproducible using seed

SEED = 42
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

np.random.seed(SEED)
random.seed(SEED)

In [117]:
# Extract all the images from drive to my folder

if not (os.path.exists('./images')):
  !cp '/content/drive/MyDrive/images.zip' . # Using cp extract the file from google drive to a local folder
  with zipfile.ZipFile('images.zip', 'r') as f:
    f.extractall('./images')

In [118]:
class Img_Dataset(Dataset):
  def __init__(self, df, transform=None, img_dir='./images'):
    self.file_names = df['image'].tolist()
    self.labels = df['class'].tolist()
    self.img_dir = img_dir
    self.transform = transform

  def __len__(self):
    return len(self.file_names)

  def __getitem__(self, idx):
    img_path = os.path.join(self.img_dir, self.file_names[idx])
    image = Image.open(img_path).convert('RGB')
    label = self.labels[idx]

    if self.transform:
      image = self.transform(image)

    return image, label, self.file_names[idx]

In [119]:
# Taking some part of the data for validation, as we don't necessarily need it
# (Since there is a lot of training data)

train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    random_state = SEED
)

In [120]:
transformer = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],
                [0.229,0.224,0.225])
])

In [121]:
train_dataset = Img_Dataset(
    train_df,
    transform = transformer)
val_dataset = Img_Dataset(
    val_df,
    transform = transformer)

In [122]:
# Creating a loder to load all the data

# Trying different batches starting off with 8, since we have a lot of data, but still the dataset isn't that large

BATCH = 8

train_loader = DataLoader(
    train_dataset,
    batch_size = BATCH,
    shuffle = True, # Shuffling data when loading it
    num_workers = 2, # Put 2 workers, so that process goes faster
    drop_last = True # Drop everything that remains after placing data into batches of 8
)

val_loader = DataLoader(
    val_dataset,
    batch_size = BATCH,
    shuffle = True,
    num_workers = 2,
    drop_last = True
)

# ***Use Pretrained ResNet50 Model as BackBone***

In [123]:
def prepare_model():

    # Load a pretrained ResNet50 backbone
    m = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)

    # Getting how many features were input for the lasat layer
    in_features = m.fc.in_features

    # Replace the 1000-class head with a single-logit head
    m.fc = nn.Linear(in_features, 1)
    return m


model = prepare_model()

# ***Preparing Everything Needed for Training***

In [124]:

# I need a balanced loss function, since the dataset is balanced (ig) (haven't taken a look at it yet tbh)
criterion = nn.BCEWithLogitsLoss()

# Use the common, simple Adam optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Use scheduler to regulate lr
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer)

# Use scaler to reduce the memory usage
scaler = torch.amp.GradScaler(DEVICE)

In [125]:
def train(model, loader, optimizer, scaler, epochs):
  model.train()
  model.to(DEVICE)


  for epoch in range(epochs):
    print(f'Epoch {epoch}')

    running_loss = 0.0
    correct = 0.0

    for imgs, labels, _ in loader:
      imgs = imgs.to(DEVICE)
      labels = labels.to(DEVICE)

      # Reshape labels to match output shape and cast to float
      labels = labels.view(-1, 1).float()

      with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda")):
        outputs = model(imgs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        # Only scale if using GPU
        if DEVICE == "cuda":
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()

        running_loss += loss.item()

        # For binary classification with BCEWithLogitsLoss, use sigmoid and threshold for predictions
        preds = torch.sigmoid(outputs) > 0.5
        correct += (labels == preds).sum().item()


    running_loss /= len(loader)
    accuracy = 100. * correct / len(loader.dataset)

    print(f'Training Loss: {running_loss:.4f}')
    print(f'Training Accuracy: {accuracy:.2f}%')


  return model, running_loss, accuracy

# ***Training***

In [126]:
model, train_loss, train_acc = train(
    model,
    train_loader,
    optimizer,
    scaler,
    10
)

Epoch 0


/tmp/ipython-input-134103086.py:19: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda")):


Training Loss: 0.1083
Training Accuracy: 92.96%
Epoch 1
Training Loss: 0.0475
Training Accuracy: 97.04%
Epoch 2
Training Loss: 0.1304
Training Accuracy: 94.81%
Epoch 3
Training Loss: 0.0346
Training Accuracy: 97.04%
Epoch 4
Training Loss: 0.0489
Training Accuracy: 97.04%
Epoch 5
Training Loss: 0.0784
Training Accuracy: 94.44%
Epoch 6
Training Loss: 0.0165
Training Accuracy: 97.78%
Epoch 7
Training Loss: 0.0053
Training Accuracy: 97.78%
Epoch 8
Training Loss: 0.0016
Training Accuracy: 97.78%
Epoch 9
Training Loss: 0.0005
Training Accuracy: 97.78%


Solid results on training

# ***Validation***

In [129]:
@torch.no_grad()
def val_report(model, loader, threshold):
    model.eval()
    all_probs, all_lbls = [], []
    for imgs, labels, _ in loader:
        imgs = imgs.to(DEVICE)
        outputs = model(imgs)
        probs = torch.sigmoid(outputs).squeeze(1).detach().cpu().numpy()
        all_probs.append(probs)
        all_lbls.extend(labels.numpy())
    all_probs = np.concatenate(all_probs)
    preds = (all_probs >= threshold).astype(int)
    print("F1:", f1_score(all_lbls, preds))


val_report(
    model,
    val_loader,
    0.5
)

F1: 1.0


Damn, this is high

# ***Save This Model***

In [132]:
if not (os.path.isfile('model.pth')):
  torch.save(model.state_dict(), 'model.pth')